In [ ]:
# Perform Google Colab installs (if running in Google Colab)
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference
    !pip install sentence-transformers # for embedding models

In [1]:
import fitz
from tqdm.auto import tqdm # for progress bars, requires !pip install tqdm"
import re
import pandas as pd
import random
import numpy as np
import torch

# Step 4: Ingesting the PDF, preprocessing the text into chunks

Take the output from step 3: Load it to notebook

In [2]:
pdf_path = 'utitlites.pdf'

### Step 4A: Parse the utitlities.pdf get the metadata

In [3]:

def parse_metadata(text):
    """Uses regex to find and parse metadata that matches the specific pattern."""
    metadata_regex = r"Ticker: (\w+), Sector: ([\w\s]+), Filed At: (\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}-\d{2}:\d{2})"
    match = re.search(metadata_regex, text)
    if match:
        return {
            "ticker": match.group(1),
            "sector": match.group(2),
            "filing_date": match.group(3)
        }
    return {"ticker": "", "sector": "", "filing_date": ""}

def text_formatter(text):
    """Replaces newlines with spaces and strips unnecessary whitespace."""
    return text.replace('\n', ' ').strip()

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, collects statistics,
    and parses metadata from the first line that matches a specific pattern.
    """
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in enumerate(doc):
        text = page.get_text()
        formatted_text = text_formatter(text)

        # Extract and parse metadata using the updated function
        metadata = parse_metadata(formatted_text)

        pages_and_texts.append({
            "page_number": page_number,
            "page_char_count": len(formatted_text),
            "page_word_count": len(formatted_text.split()),
            "page_sentence_count_raw": len(formatted_text.split(". ")),
            "page_token_count": len(formatted_text.split()) / 4,
            "ticker": metadata['ticker'],
            "sector": metadata['sector'],
            "filing_date": metadata['filing_date'],
            "text": formatted_text
        })
    return pages_and_texts


### Output of 4A: List of dictionaries

In [4]:
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)

In [6]:
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,ticker,sector,filing_date,text
0,0,5097,743,34,185.75,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023..."
1,1,3631,513,19,128.25,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023..."
2,2,3621,502,17,125.50,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023..."
3,3,3674,540,19,135.00,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023..."
4,4,3719,574,19,143.50,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023..."


## Step 4B: Split the text into sentences using sPacy

In [8]:
from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()
nlp.add_pipe("sentencizer")

In [9]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1494 [00:00<?, ?it/s]

## Output of step 4B: Split the text into sentences using sPacy

In [10]:
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,ticker,sector,filing_date,text,sentences,page_sentence_count_spacy
0,0,5097,743,34,185.75,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023...","[Ticker: LNT, Sector: Utilities, Filed At: 202...",32
1,1,3631,513,19,128.25,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023...","[Ticker: LNT, Sector: Utilities, Filed At: 202...",19
2,2,3621,502,17,125.50,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023...","[Ticker: LNT, Sector: Utilities, Filed At: 202...",17
3,3,3674,540,19,135.00,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023...","[Ticker: LNT, Sector: Utilities, Filed At: 202...",19
4,4,3719,574,19,143.50,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023...","[Ticker: LNT, Sector: Utilities, Filed At: 202...",19


## Step 4C: Group Sentences into chunks, all-mpnet-base-v2
By default, input text longer than 384 word pieces is truncated

In [12]:
import re
from tqdm import tqdm

# Assuming an average of 1 token = 4 characters, but you'll adjust based on your actual tokenization logic
TOKEN_CHAR_RATIO = 4
MAX_TOKEN_COUNT = 384  # Maximum token count per chunk

def split_list_by_token_count(sentences: list[str], max_token_count: int) -> list[list[str]]:
    """
    Splits a list of sentences into sublists, ensuring the total token count of each sublist
    is as close as possible to the max_token_count without exceeding it.
    """
    chunks = []
    current_chunk = []
    current_char_count = 0

    for sentence in sentences:
        sentence_char_count = len(sentence)
        if current_char_count + sentence_char_count <= max_token_count * TOKEN_CHAR_RATIO:
            current_chunk.append(sentence)
            current_char_count += sentence_char_count
        else:
            chunks.append(current_chunk)
            current_chunk = [sentence]
            current_char_count = sentence_char_count
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

# Now, instead of a fixed number of sentences per chunk, split based on token count
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    # Dynamically split sentences into chunks based on token count
    item["sentence_chunks"] = split_list_by_token_count(item["sentences"], MAX_TOKEN_COUNT)

    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        chunk_dict["page_number"] = item["page_number"]
        chunk_dict["ticker"] = item["ticker"]
        chunk_dict["sector"] = item["sector"]
        chunk_dict["filing_date"] = item["filing_date"]

        # Join sentences to form a paragraph-like chunk
        joined_sentence_chunk = " ".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)  # Improving punctuation spacing

        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len(joined_sentence_chunk.split(" "))
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / TOKEN_CHAR_RATIO  # Approximate token count

        pages_and_chunks.append(chunk_dict)

# Display the total number of chunks created
total_chunks = len(pages_and_chunks)
total_chunks


100%|██████████| 1494/1494 [00:00<00:00, 14576.19it/s]


4497

## Output of Step 4C:


In [13]:
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.head()

,page_number,ticker,sector,filing_date,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,0,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023...",1458,212,364.50
1,0,LNT,Utilities,2023-02-24T16:03:25-05:00,2) WPL - is a public utility engaged principal...,1298,190,324.50
2,0,LNT,Utilities,2023-02-24T16:03:25-05:00,TABLE_START 3 ##TABLE_ENDTable of Co ntents Co...,1376,204,344.00
3,0,LNT,Utilities,2023-02-24T16:03:25-05:00,"Employees - At December 31, 2022, Alliant Ener...",914,138,228.50
4,1,LNT,Utilities,2023-02-24T16:03:25-05:00,"Ticker: LNT, Sector: Utilities, Filed At: 2023...",1415,200,353.75
...,...,...,...,...,...,...,...,...
4492,1491,XEL,Utilities,2023-02-23T14:54:01-05:00,Changes in environmental policies and regulati...,745,115,186.25
4493,1492,XEL,Utilities,2023-02-23T14:54:01-05:00,"Ticker: XEL, Sector: Utilities, Filed At: 2023...",1470,219,367.50
4494,1492,XEL,Utilities,2023-02-23T14:54:01-05:00,We establish strategies and expectations relat...,1494,209,373.50
4495,1492,XEL,Utilities,2023-02-23T14:54:01-05:00,"More frequent and severe drought conditions, e...",675,96,168.75


## Step 4D: Remove chunks that contain little to no information:

In [14]:
# Show random chunks with under 20 tokens in length
min_token_length = 20
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 16.75 | Text: Ticker: NJR, Sector: Utilities, Filed At: 2023-02-09T19:12:06-05:00


In [15]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:1]

[{'page_number': 0,
  'ticker': 'LNT',
  'sector': 'Utilities',
  'filing_date': '2023-02-24T16:03:25-05:00',
  'sentence_chunk': 'Ticker: LNT, Sector: Utilities, Filed At: 2023-02-24T16:03:25-05:00 ITEM 1. BUSINESS A. GENERAL Alliant Energy maintains its principal executive offices in Madison, Wisconsin. Alliant Energy operates as a regulated investor-owned public utility holding company, and its purpose-driven strategy is to serve its customers and build stronger communities. Alliant Energys primary focus is to provide regulated electric and natural gas service to approximately 995,000 electric and approximately 425,000 natural gas customers in the Midwest through its two public utility subsidiaries, IPL and WPL. The primary first tier wholly-owned subsidiaries of Alliant Energy are as follows: 1) IPL - is a public utility engaged principally in the generation and distribution of electricity and the distribution and transportation of natural gas to retail customers in select markets 

# Step 4E: Save the embeddings

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
%%time
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")
# Send the model to the GPU
embedding_model.to("cuda") # requires a GPU installed, for reference on my local machine, I'm using a NVIDIA RTX 4090

# Create embeddings one by one on the GPU
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
100%|██████████| 4475/4475 [00:59<00:00, 74.79it/s]

CPU times: user 3min 6s, sys: 1.08 s, total: 3min 7s
Wall time: 1min 5s


In [17]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)